In [1]:
!pip install fastapi uvicorn nest_asyncio pyngrok transformers torch sacremoses --quiet

In [2]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, T5ForConditionalGeneration
from fastapi import FastAPI, Form
from pydantic import BaseModel
from fastapi.responses import HTMLResponse
import nest_asyncio
import uvicorn
from pyngrok import ngrok


In [3]:
# Khởi tạo BioGPT và mô hình dịch
bio_tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT")
bio_model = AutoModelForCausalLM.from_pretrained("microsoft/BioGPT")
translation_tokenizer = AutoTokenizer.from_pretrained("VietAI/envit5-translation")
translation_model = T5ForConditionalGeneration.from_pretrained("VietAI/envit5-translation")

# Kiểm tra thiết bị và chuyển mô hình sang GPU nếu có
device = "cuda" if torch.cuda.is_available() else "cpu"
bio_model = bio_model.to(device)
translation_model = translation_model.to(device)



config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.49M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

In [4]:
# Khởi tạo ứng dụng FastAPI
app = FastAPI()

In [5]:
# Hàm dịch
def translate(text, direction="vi-en"):
    prompt = "translate Vietnamese to English: " if direction == "vi-en" else "translate English to Vietnamese: "
    inputs = translation_tokenizer(prompt + text, return_tensors="pt").to(device)
    outputs = translation_model.generate(
        **inputs,
        max_length=2048,
        num_beams=5,  # Tăng lên từ 5-10 để cải thiện chất lượng
        early_stopping=True
    )

    return translation_tokenizer.decode(outputs[0], skip_special_tokens=True)



In [6]:
!ngrok authtoken 2oPcvV2YiJScnYzCORQaPbuHP5q_4Q8A8E6j6V8vEMQpMJrVr

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
# Trang chủ với form nhập liệu đơn giản
@app.get("/", response_class=HTMLResponse)
async def home():
    return """
    <html>
        <head>
            <title>BioGPT API</title>
        </head>
        <body>
            <h1>BioGPT API</h1>
            <form action="/process_biogpt" method="post">
                <label for="sentence">Nhập câu cần xử lý:</label><br>
                <input type="text" id="sentence" name="sentence" style="width: 300px;"><br><br>
                <label for="translate_to_vietnamese">Sử dụng tiếng Việt:</label>
                <input type="checkbox" id="translate_to_vietnamese" name="translate_to_vietnamese"><br><br>
                <input type="submit" value="Gửi yêu cầu">
            </form>
        </body>
    </html>
    """



In [8]:
# Endpoint để xử lý câu và trả về kết quả từ BioGPT
@app.post("/process_biogpt", response_class=HTMLResponse)
async def process_biogpt(
    sentence: str = Form(...),
    translate_to_vietnamese: bool = Form(False)
):
    # Chuẩn bị đầu vào cho BioGPT
    if translate_to_vietnamese:
        sentence = translate(sentence, "en-vi")
    inputs = bio_tokenizer(sentence, return_tensors="pt").to(device)
    with torch.no_grad():
        beam_output = bio_model.generate(
            **inputs, min_length=100, max_length=1024, num_beams=5,
            early_stopping=True, temperature=0.3, top_p=0.8, repetition_penalty=2.0,
            no_repeat_ngram_size=5, do_sample=True
        )
    output_text = bio_tokenizer.decode(beam_output[0], skip_special_tokens=True)

    # Dịch lại nếu có chọn tùy chọn
    if translate_to_vietnamese:
        output_text = translate(output_text, "en-vi")

    # Trả về trang HTML với kết quả
    return f"""
    <html>
        <body>
            <h2>Kết quả từ BioGPT:</h2>
            <p>{output_text}</p>
            <a href="/">Quay lại</a>
        </body>
    </html>
    """



In [9]:
# Chạy server với ngrok
if __name__ == "__main__":
    # Cho phép async trong notebook
    nest_asyncio.apply()
    
    # Khởi tạo ngrok tunnel
    public_url = ngrok.connect(8000)
    print("Public URL:", public_url)

    # Chạy server
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [23]


INFO:     Waiting for application startup.


INFO:     Application startup complete.


INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://cdcd-34-44-37-82.ngrok-free.app" -> "http://localhost:8000"


...

INFO:     42.118.45.46:0 - "GET / HTTP/1.1" 200 OK


INFO:     42.118.45.46:0 - "POST /process_biogpt HTTP/1.1" 200 OK


INFO:     42.118.45.46:0 - "GET / HTTP/1.1" 200 OK


INFO:     42.118.45.46:0 - "POST /process_biogpt HTTP/1.1" 200 OK


INFO:     42.118.45.46:0 - "GET / HTTP/1.1" 200 OK
